# TM10007 Assignment template

In [ ]:
# Run this to use from colab environment
# !pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

  Preparing metadata (setup.py) ... done


# Importing packages

In [30]:
# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn

# Classifiers
from sklearn import model_selection
from sklearn import metrics

# Preprocessing steps
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from scipy.stats import shapiro

# Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [82]:
## Data loading functions. Uncomment the one you want to use
# from worcgist.load_data import load_dat
from worclipo.load_data import load_data
#from worcliver.load_data import load_data
#from ecg.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')



The number of samples: 115
The number of columns: 494


## Creating a split in the data

A training and test set are created. The training set will be used to fit the classifier, while the test set is remained untouched until it is used for testing the trained classifier.

Split is stratified so labels are present in both sets in same proportions.

In [83]:
# Extract columns with labels
Y = data.pop('label').replace({'lipoma':0, 'liposarcoma':1}) #moeten we misschien nog ook de ID poppen?

# Extract the rest of the data (without the labels)
X = data

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2, stratify=Y) # ONDERBOUWEN IN REPORT

print(f'Size of training set: {X_train.shape}')
print(f'Size of test set: {X_test.shape}')

Size of training set: (92, 493)
Size of test set: (23, 493)


## Inspecting distribution of features
To determine data distribution of train set

### Visually inspect distribution of features

In [88]:
# Check if Gaussian distribution per column
# if Gaussian distribution, take mean for filling empty cells (later in script)
# if not Gaussion distribution (and outliers or skewed), take median for filling empty cells (later in script)

# plt.hist(X_train.iloc[:, 15], bins=30, edgecolor='k') # Change number of colum to see distribution (between width dataframe)
plt.hist(X_train['PREDICT_original_tf_Gabor_entropy_F0.2_A1.57'], bins=30, edgecolor='k') # Change number of colum to see distribution (between width dataframe)


plt.title('Distribution of Feature')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

# print(X_train.head())

### Determining data distribution of train set

In [86]:
# Definieer alpha voor de significantietest BEARGUMENTEREN WAAROM DEZE TEST IN REPORT + WAARDE
alpha = 0.05

# Initialiseer een teller voor normaal verdeelde kolommen
normal_distributed_columns_count = 0

# Loop door elke kolom in de DataFrame en voer Shapiro-Wilk test uit
for column in X_train:
    stat, p = shapiro(X_train[column])
    if p > alpha:
        # print(column)
        normal_distributed_columns_count += 1  # Tel kolom als normaal verdeeld

print(f"Aantal normaal verdeelde kolommen: {normal_distributed_columns_count}")


PREDICT_original_sf_compactness_avg_2.5D
PREDICT_original_sf_prax_avg_2.5D
PREDICT_original_sf_prax_std_2.5D
PREDICT_original_hf_skewness
PREDICT_original_hf_entropy
PREDICT_original_logf_peak_position_sigma1
PREDICT_original_logf_entropy_sigma1
PREDICT_original_logf_min_sigma5
PREDICT_original_logf_std_sigma5
PREDICT_original_logf_peak_position_sigma5
PREDICT_original_logf_range_sigma5
PREDICT_original_logf_entropy_sigma5
PREDICT_original_logf_peak_position_sigma10
PREDICT_original_logf_entropy_sigma10
PREDICT_original_tf_LBP_min_R3_P12
PREDICT_original_tf_LBP_max_R3_P12
PREDICT_original_tf_LBP_range_R3_P12
PREDICT_original_tf_LBP_entropy_R3_P12
PREDICT_original_tf_LBP_max_R8_P24
PREDICT_original_tf_LBP_peak_R8_P24
PREDICT_original_tf_LBP_peak_position_R8_P24
PREDICT_original_tf_LBP_entropy_R8_P24
PREDICT_original_tf_LBP_max_R15_P36
PREDICT_original_tf_LBP_entropy_R15_P36
PREDICT_original_tf_GLCM_homogeneityd3.0A0.79
PREDICT_original_tf_GLCMMS_homogeneityd1.0A1.57std
PREDICT_original_

In [53]:
# print(X_train.head())
# print(Y.head())

## Preprocessing Data

### Missing Values
Determining cells with missing values and filling these

In [89]:
# Check missing values
missing_values = X_train.isnull().sum()
print(len(missing_values[missing_values > 0]))

0


In [81]:
# Filling missing values with correct value based on type of distribution per column

# Definieer alpha voor de significantietest BEARGUMENTEREN WAAROM DEZE TEST IN REPORT + WAARDE
alpha = 0.05

# Initialiseer een teller voor normaal verdeelde kolommen
normal_distributed_columns_count = 0
not_normal_distributed_columns_count = 0
nans_found_in_column_count = 0

# Loop door elke kolom in de DataFrame en voer Shapiro-Wilk test uit
for column in X_train:
    stat, p = shapiro(X_train[column])
    if p > alpha:
        # print(column)
        normal_distributed_columns_count += 1  # Tel kolom als normaal verdeeld
        if X_train[column].isna().sum() > 0:
            X_train[column].fillna(X_train[column].mean)
            nans_found_in_column_count += 1
    else:
        not_normal_distributed_columns_count += 1  # Tel kolom als normaal verdeeld
        if X_train[column].isna().sum() > 0:
            X_train[column].fillna(X_train[column].median)
            nans_found_in_column_count += 1

print(f'The number of normal distributed columns is: {normal_distributed_columns_count}')
print(f'The number of not normal distributed columns is: {not_normal_distributed_columns_count}')
print(f'The number of columns where nans were filled: {nans_found_in_column_count}')

The number of normal distributed columns is: 89
The number of not normal distributed columns is: 404
The number of columns where nans were filled: 0


### Feature Scaling
Scale features in Train set

# Inspecting data

In [ ]:
## uncomment following lines to print different info
# print(X_train.head())
# print(X_train.info())
# print(X_train.describe()) # Gives 8 metrics per column [8x494]



# Pipeline for numerical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values
    ('scaler', StandardScaler()),  # Scale features
    ('pca', PCA(n_components=0.95))  # Reduce dimensionality (nog kijken welke waarden)
])

X_preprocessed = numeric_transformer.fit_transform(X_train)
print(X_preprocessed)

Series([], dtype: int64)
[[  3.69308699  10.49965385   8.12652172 ...  -0.14012688  -0.18474327
    1.24139976]
 [ -4.56127894   2.81882473  -7.840514   ...   1.63287141   1.38134925
   -1.52803909]
 [ -0.74152394   8.56443472   0.86427269 ...   0.10569603   1.22296131
   -0.7835249 ]
 ...
 [ -4.07252116   3.20510508   7.39819921 ...  -0.56923872  -0.41247159
   -2.87638164]
 [  2.92986516  -5.37845493   7.16185485 ...   1.10432961  -0.65658447
    1.31493125]
 [-11.36513284 -13.00480223  -7.77865287 ...   1.82404703   0.12141449
   -1.78605994]]
